# Big route model 2

A big model for the route 15

We will have all of the usual features + these two.

a) The time of the bus leaving the first stop on the route
b) the distance of the bus stop from the first stop on the route

StopA, stopB won't be features in the model. Rather, we will be predicting the time for the bus to get to each successive stop on the route. e.g

Time from stop 1 to stop 2
Time from stop 1 to stop 3
Time from stop 1 to stop 4
Time from stop 1 to stop N

## First gather all the data for the route

In [1]:
# Get data for route 15
import json
routes = json.loads(open('/home/student/dbanalysis/dbanalysis/resources/trimmed_routes.json','r').read())

In [2]:
r=routes['15'][1]

In [3]:
from dbanalysis import stop_tools
import pandas as pd

In [4]:
to_concat=[]
for i in range(1, len(r)-1):
    a=r[i]
    b=r[i+1]
    df=stop_tools.get_stop_link(a,b,merge_weather=True)
    to_concat.append(df)
    del(df)
    
df = pd.concat(to_concat,axis=0)
    

18392 18380
18518 18507
18744 18733
18915 18903
18961 18949
34833 34812
35092 35071
49594 49564
33441 33419
33381 33356
33387 33359
49987 49950
50215 50173
50184 50140
50096 50053
68681 68627
68817 68761
71444 71383
68799 68733
80965 80885
81013 80929
80983 80893
81047 80952
81106 80998
126712 126539
126531 126353
129054 128860
129063 128856
129012 128789
129256 129016
56482 56450
30211 30191
6342 6341
556 556
66391 66355
66357 66319
66284 66245
88438 88386
89505 89446
89653 89591
89768 89705
89755 89690
89853 89786
75706 75645
61041 60987
61090 61029
61134 61064
61082 61006
37441 37380
37430 37356
37429 37351
26335 26285
34030 33968
34027 33954
34035 33959
34129 34049
34088 34000
34086 33986
26293 26218
18502 18411
18536 18428
18531 18413
18553 18413
18565 18419
18578 18416
18475 18289
18539 18323
18447 18235
18395 18186


In [5]:
del(to_concat)

In [7]:
af=df[df['routeid'].isin(['15_16','15_10'])]

In [9]:
del(df)

In [12]:
af.shape

(1262844, 19)

In [6]:
df['routeid'].unique()

array(['15_16', '15_10', '27X_42', '27_19', '27_20', '42_44', '43_89',
       '43_88', '27B_34', '27A_5', '27B_38', '27B_36', '27B_35', '27B_39',
       '27B_37', '27A_6', '104_16', '14_15', '14C_17', '29A_15', '32_58',
       '130_11', '31_18', '31A_26', '31B_45', '31B_46', '53_19', '151_15',
       '151_16', '142_10', '15A_81', '15B_55', '140_19', '15A_80',
       '15B_56', '15B_57', '65B_65', '83_16', '65_72', '65_74', '83A_17',
       '65_73', '83A_20', '83_19', '142_8', '49_27'], dtype=object)

In [7]:
data = df[df['routeid'].isin(['15_16', '15_10'])]

In [8]:
del(df)

In [9]:
df=data

In [10]:
df=df.dropna()

In [11]:
df.columns

Index(['dayofservice', 'tripid', 'plannedtime_arr_from',
       'plannedtime_dep_from', 'actualtime_arr_from', 'actualtime_dep_from',
       'plannedtime_arr_to', 'actualtime_arr_to', 'routeid', 'stopA', 'stopB',
       'hour', 'dewpt', 'msl', 'rain', 'rhum', 'temp', 'vappr', 'wetb'],
      dtype='object')

In [6]:
print('hello')

hello


## Now get the distances from the first stop on the route

In [12]:
s_getter =stop_tools.stop_getter()
total_distance = 0
route_distances = {r[1]:0}
for i in range(1, len(r)-1):
    distance = s_getter.get_stop_distance(str(r[i]),str(r[i+1]))
    
    total_distance += distance
    route_distances[r[i+1]]=total_distance
    

In [13]:
df['distance']=df['stopA'].apply(lambda x: route_distances[x])

## Now add the time of leaving the first stop

In [14]:
keys= df[df['stopA']==6318]
keys['base_time_dep']=keys['actualtime_arr_from']
keys2=keys[['tripid','dayofservice','base_time_dep']]
full_df = pd.merge(df,keys2,on=['dayofservice','tripid'])

/home/student/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [65]:
del (df)
del (data)

## Now add time information

In [15]:
full_df['traveltime']=full_df['actualtime_arr_from']-full_df['base_time_dep']

In [17]:
time_format = "%d-%b-%y %H:%M:%S"
full_df['dt']=pd.to_datetime(full_df['dayofservice'],format=time_format)
full_df['day']=full_df['dt'].dt.dayofweek
full_df['month']=full_df['dt'].dt.month
full_df['weekend']=full_df['day']>4
full_df['year']=full_df['dt'].dt.year

In [18]:
full_df.columns

Index(['dayofservice', 'tripid', 'plannedtime_arr_from',
       'plannedtime_dep_from', 'actualtime_arr_from', 'actualtime_dep_from',
       'plannedtime_arr_to', 'actualtime_arr_to', 'routeid', 'stopA', 'stopB',
       'hour', 'dewpt', 'msl', 'rain', 'rhum', 'temp', 'vappr', 'wetb',
       'distance'],
      dtype='object')

In [19]:
len(full_df)

1262844

In [21]:
full_df=full_df[full_df['traveltime']!=0]

In [22]:
len(full_df)

1170823

In [23]:
full_df.columns

Index(['dayofservice', 'tripid', 'plannedtime_arr_from',
       'plannedtime_dep_from', 'actualtime_arr_from', 'actualtime_dep_from',
       'plannedtime_arr_to', 'actualtime_arr_to', 'routeid', 'stopA', 'stopB',
       'hour', 'dewpt', 'msl', 'rain', 'rhum', 'temp', 'vappr', 'wetb',
       'distance', 'base_time_dep', 'traveltime', 'dt', 'day', 'month',
       'weekend', 'year'],
      dtype='object')

# Now we have a dataframe for route 15
lets define some feature sets

In [24]:
simple_features_A = ['hour','day','base_time_dep','weekend','distance']

In [37]:
train = full_df[full_df['year']==2016]
test = full_df[full_df['year']==2017]

# Model with linear regression

In [25]:
from sklearn.linear_model import LinearRegression as lr

In [27]:
rgr = lr(fit_intercept=True).fit(train[simple_features_A],train['traveltime'])

In [28]:
preds_A = rgr.predict(train[simple_features_A])

In [29]:
from sklearn import metrics

In [30]:
metrics.r2_score(train['traveltime'],preds_A)

0.9247493774959163

In [31]:
((abs(train['traveltime']-preds_A)/train['traveltime'])*100).mean()

41.993757072313144

In [38]:
# test on just one feature (time for whole route)
test2 = test[test['distance']==test['distance'].max()]
preds_B = rgr.predict(test2[simple_features_A])

In [40]:
metrics.r2_score(test2['traveltime'],preds_B)

0.22963261381217104

In [41]:
((abs(test2['traveltime']-preds_B)/test2['traveltime'])*100).mean()

11.69507533905664

# So even using simple, simple linear regression, the mean error for the whole route isn't thaaaaat bad, not really.

In [43]:
#more complex features
dummy_df = pd.get_dummies(full_df,columns=['day','hour','month'])

In [44]:
dummy_df.columns

Index(['dayofservice', 'tripid', 'plannedtime_arr_from',
       'plannedtime_dep_from', 'actualtime_arr_from', 'actualtime_dep_from',
       'plannedtime_arr_to', 'actualtime_arr_to', 'routeid', 'stopA', 'stopB',
       'dewpt', 'msl', 'rain', 'rhum', 'temp', 'vappr', 'wetb', 'distance',
       'base_time_dep', 'traveltime', 'dt', 'weekend', 'year', 'day_0',
       'day_1', 'day_2', 'day_3', 'day_4', 'day_5', 'day_6', 'hour_5',
       'hour_6', 'hour_7', 'hour_8', 'hour_9', 'hour_10', 'hour_11', 'hour_12',
       'hour_13', 'hour_14', 'hour_15', 'hour_16', 'hour_17', 'hour_18',
       'hour_19', 'hour_20', 'hour_21', 'hour_22', 'hour_23', 'month_1',
       'month_2', 'month_3', 'month_4', 'month_5', 'month_6'],
      dtype='object')

In [46]:
simple_features_B = ['base_time_dep', 'weekend','distance','day_0',
       'day_1', 'day_2', 'day_3', 'day_4', 'day_5', 'day_6', 'hour_5',
       'hour_6', 'hour_7', 'hour_8', 'hour_9', 'hour_10', 'hour_11', 'hour_12',
       'hour_13', 'hour_14', 'hour_15', 'hour_16', 'hour_17', 'hour_18',
       'hour_19', 'hour_20', 'hour_21', 'hour_22', 'hour_23', 'month_1',
       'month_2', 'month_3', 'month_4', 'month_5', 'month_6']

In [52]:
train = dummy_df[dummy_df['year'] == 2016]
test = dummy_df[dummy_df['year']==2017]

In [53]:
rgr = lr(fit_intercept=True).fit(train[simple_features_B],train['traveltime'])

In [55]:
preds_C = rgr.predict(train[simple_features_B])

In [56]:
metrics.r2_score(train['traveltime'],preds_C)

0.9378605100350959

In [58]:
((abs(train['traveltime']-preds_C)/train['traveltime'])*100).mean()

45.026134732203204

In [59]:
# test on just one feature (time for whole route)
test2 = test[test['distance']==test['distance'].max()]
preds_B = rgr.predict(test2[simple_features_B])

In [60]:
metrics.r2_score(test2['traveltime'],preds_B)

0.4089228642520375

In [61]:
((abs(test2['traveltime']-preds_B)/test2['traveltime'])*100).mean()

10.231348275023402

# So this model is quite good at predicting for the final stop on the route, but likely terrible at predicting for those in between

Now test for all of the distances in the route...

We get a meaningless r2 score, and a very high MAPE

In [62]:
preds_B = rgr.predict(test[simple_features_B])

In [63]:
metrics.r2_score(test['traveltime'],preds_B)

0.93601052474354

In [64]:
((abs(test['traveltime']-preds_B)/test['traveltime'])*100).mean()

45.899750813767504

# add weather

In [66]:
simple_features_C = simple_features_B + ['rain', 'temp']

In [67]:
rgr = lr(fit_intercept=True).fit(train[simple_features_C],train['traveltime'])

In [69]:
preds_B = rgr.predict(test[simple_features_C])

In [70]:
metrics.r2_score(test['traveltime'],preds_B)

0.9357665959238073

In [71]:
((abs(test['traveltime']-preds_B)/test['traveltime'])*100).mean()

46.29122545298618

In [73]:
# test on just one feature (time for whole route)
test2 = test[test['distance']==test['distance'].max()]
preds_B = rgr.predict(test2[simple_features_C])

In [76]:
metrics.r2_score(test2['traveltime'],preds_B)

0.41118333530778006

In [77]:
((abs(test2['traveltime']-preds_B)/test2['traveltime'])*100).mean()

10.19576390318248

# Try other models.

# Model 1. Random Forest

In [78]:
from sklearn.ensemble import RandomForestRegressor as rfr

In [80]:
rgr = rfr().fit(train[simple_features_C],train['traveltime'])

In [81]:
preds_B = rgr.predict(train[simple_features_C])

In [82]:
metrics.r2_score(train['traveltime'],preds_B)

0.9984505416788239

In [83]:
((abs(train['traveltime']-preds_B)/train['traveltime'])*100).mean()

2.684655513443022

In [84]:
preds_C = rgr.predict(test[simple_features_C])

In [86]:
metrics.r2_score(test['traveltime'],preds_C)

0.9600555887198465

In [87]:
((abs(test['traveltime']-preds_C)/test['traveltime'])*100).mean()

10.389133366871654

## Then just for one trip


In [88]:
test2 = test[test['distance']==test['distance'].max()]

In [90]:
preds_C = rgr.predict(test2[simple_features_C])

In [91]:
metrics.r2_score(test2['traveltime'],preds_C)

0.5703429732391834

In [92]:
((abs(test2['traveltime']-preds_C)/test2['traveltime'])*100).mean()

7.723682633043119

In [93]:
test2 = test[test['distance']==test['distance'].median()]

In [95]:
preds_C = rgr.predict(test2[simple_features_C])

In [96]:
metrics.r2_score(test2['traveltime'],preds_C)

0.4545912622559859

In [97]:
((abs(test2['traveltime']-preds_C)/test2['traveltime'])*100).mean()

9.082183275815371

In [98]:
metrics.mean_absolute_error(test2['traveltime'],preds_C)

245.4642078310301

Now for the first stop on the trip

In [99]:
test['distance'].unique()

array([ 0.19439372,  0.42661594,  0.91399099,  2.12746783,  2.13710899,
        2.41355696,  2.73836243,  3.05509448,  3.47239308,  3.98918733,
        4.29713731,  4.60077595,  5.07080026,  5.22224552,  5.52456992,
        5.73902979,  6.13097876,  6.47090175,  6.80926407,  7.12874632,
        7.41718128,  7.73113298,  8.15250246,  8.459664  ,  8.79233752,
        9.06522538,  9.34944473,  9.63199619,  9.91663643, 10.07682693,
       10.38606964, 11.79891293, 12.14116219, 12.56124053, 13.00315193,
       13.38822997, 13.59259852, 13.8187145 , 13.9437401 , 14.10590383,
       14.33519184, 14.54172718, 14.83736879, 15.17371397, 15.38787347,
       15.67475222, 15.87786843, 16.06248638, 16.37022232, 16.61976575,
       16.90819453, 17.16837317, 17.39439661, 17.86033059, 18.30079427,
       18.76220611, 19.16280228, 19.600695  , 19.86110042, 20.46557701,
       20.63406303, 20.9327889 , 21.35775595, 22.17246364, 22.28254239,
       22.60855934, 11.24524183, 11.02329844])

In [104]:
test3 = test[(test['distance']>0) & ( test['distance']<0.2)]

In [105]:
len(test3)

7196

In [106]:
test3['distance'].unique()

array([0.19439372])

In [107]:
preds_C = rgr.predict(test3[simple_features_C])

In [108]:
metrics.r2_score(test3['traveltime'],preds_C)

-0.401011274376323

In [109]:
((abs(test3['traveltime']-preds_C)/test3['traveltime'])*100).mean()

29.843328080637185

# So for just predicting the second stop on the trip, the metrics are abysmal. Awful, really

In [110]:
test['distance'].unique()

array([ 0.19439372,  0.42661594,  0.91399099,  2.12746783,  2.13710899,
        2.41355696,  2.73836243,  3.05509448,  3.47239308,  3.98918733,
        4.29713731,  4.60077595,  5.07080026,  5.22224552,  5.52456992,
        5.73902979,  6.13097876,  6.47090175,  6.80926407,  7.12874632,
        7.41718128,  7.73113298,  8.15250246,  8.459664  ,  8.79233752,
        9.06522538,  9.34944473,  9.63199619,  9.91663643, 10.07682693,
       10.38606964, 11.79891293, 12.14116219, 12.56124053, 13.00315193,
       13.38822997, 13.59259852, 13.8187145 , 13.9437401 , 14.10590383,
       14.33519184, 14.54172718, 14.83736879, 15.17371397, 15.38787347,
       15.67475222, 15.87786843, 16.06248638, 16.37022232, 16.61976575,
       16.90819453, 17.16837317, 17.39439661, 17.86033059, 18.30079427,
       18.76220611, 19.16280228, 19.600695  , 19.86110042, 20.46557701,
       20.63406303, 20.9327889 , 21.35775595, 22.17246364, 22.28254239,
       22.60855934, 11.24524183, 11.02329844])

In [111]:
arr = [distance for distance in test['distance'].unique()]

In [113]:
print('hello')

hello


In [114]:
print('hello')

hello


In [116]:
arr = sorted(arr)

In [118]:
arr = [0] + arr + [23]

In [125]:
for i in range(1, len(arr)-1):
    test3 = test[(test['distance']>arr[i-1]) & (test['distance']<arr[i+1])]
    preds = rgr.predict(test3[simple_features_C])
    print('\n\n')
    print('r2:',metrics.r2_score(test3['traveltime'],preds))
    print('MAPE:',((abs(test3['traveltime']-preds)/test3['traveltime'])*100).mean())
    print('max travel time',test3['traveltime'].max())
    print('min travel time', test3['traveltime'].min())
    print('mean travel time',test3['traveltime'].mean())
    del (preds)
    del (test3)




r2: -0.401011274376323
MAPE: 29.843328080637185
max travel time 1113
min travel time 1
mean travel time 23.594219010561424



r2: -0.29968377461028495
MAPE: 27.146656832890393
max travel time 1151
min travel time 18
mean travel time 64.49831413318348



r2: -0.4209064046237443
MAPE: 18.591875587452165
max travel time 1243
min travel time 59
mean travel time 156.62443693693695



r2: 0.09161694291692368
MAPE: 16.66797654232437
max travel time 1353
min travel time 123
mean travel time 242.76107439178736



r2: 0.1972486717658587
MAPE: 14.200725458184866
max travel time 1466
min travel time 40
mean travel time 334.8658245614035



r2: 0.2505150319282422
MAPE: 14.063478889377137
max travel time 1548
min travel time 84
mean travel time 383.46099789177794



r2: 0.28753079137595283
MAPE: 13.357137348379945
max travel time 1859
min travel time 141
mean travel time 448.32776677045007



r2: 0.2968843652312897
MAPE: 13.193106816914113
max travel time 1894
min travel time 226
mean travel time




r2: 0.5738726046738318
MAPE: 7.754863139067039
max travel time 8421
min travel time 2171
mean travel time 4819.021616684427



r2: 0.5703429732391834
MAPE: 7.723682633043119
max travel time 8466
min travel time 2206
mean travel time 4869.910511148188


In [123]:
test3['traveltime'].min()

NameError: name 'test3' is not defined

In [130]:

del(RandomForestRegressor)

NameError: name 'RandomForestRegressor' is not defined

# What about a neural network?

In [128]:
from sklearn.neural_network import MLPRegressor as NN


In [132]:
rgr = NN().fit(train[simple_features_C],train['traveltime'])

/home/student/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


In [133]:
dummy_df.to_csv('/data/dummy_15_df.csv',index=False)

In [134]:
print('hello')


hello
